In [2]:


import torch
from torch import nn
import open_clip
from transformers import CLIPProcessor, CLIPModel
from transformers import AlignProcessor, AlignModel
from transformers import AltCLIPModel, AltCLIPProcessor
from transformers import FlavaProcessor, FlavaModel
from transformers import logging as transformers_logging
from transformers import ViltProcessor, ViltForImageAndTextRetrieval
from transformers import AutoProcessor, BlipModel
from transformers import FlavaProcessor, FlavaForPreTraining, BertTokenizer, FlavaFeatureExtractor
import torch.nn.functional as F
import torch

class Clip(nn.Module):
    def __init__(self, device) -> None:
        super(Clip, self).__init__()
        self.device = device

    def forward(self, image):
        pass

class Vilt(Clip):
    def __init__(self, device, model_name="dandelin/vilt-b32-finetuned-coco"):
        super(Vilt, self).__init__(device)
        self.model = ViltForImageAndTextRetrieval.from_pretrained(model_name).to(device)
        self.processor = ViltProcessor.from_pretrained(model_name)
        self.device = device

    def forward(self, image, text_list):
        with torch.no_grad():
            scores = dict()
            for text in text_list:
                # prepare inputs
                self.model.eval()
                encoding = self.processor(image, text, return_tensors="pt").to(self.device)
                outputs = self.model(**encoding)
                scores[text] = outputs.logits[0, :].item()
            output = list(F.softmax(torch.tensor(list(scores.values()))).numpy())
        return [output]


class Flava(Clip):
    def __init__(self, device, model_name="facebook/flava-full"):
        super(Flava, self).__init__(device)

        self.model = FlavaForPreTraining.from_pretrained(model_name).eval().to(device)
        self.feature_extractor = FlavaFeatureExtractor.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.processor = FlavaProcessor.from_pretrained(model_name)
        self.device = device

    def forward(self, image, text_list):
        with torch.no_grad():
            text_input = self.tokenizer(text=text_list, return_tensors="pt", padding="max_length", max_length=77).to(self.device)
            self.model.eval()
            text_feats = self.model.flava.get_text_features(**text_input).cpu().numpy()[:, 0, :]
            inputs = self.feature_extractor(images=image, return_tensors="pt").to(self.device)
            image_feats = self.model.flava.get_image_features(**inputs).cpu().numpy()[:, 0, :]
            scores = image_feats @ text_feats.T
            prob = torch.tensor(scores).softmax(dim=1).cpu().numpy()
        return prob



class Blip(Clip):
    def __init__(self, device, model_name="Salesforce/blip-image-captioning-base"):
        super(Blip, self).__init__(device)
        self.model = BlipModel.from_pretrained(model_name).to(device)
        self.processor = AutoProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt", padding=True
            ).to(self.device)
            self.model.eval()
            outputs = self.model(**inputs)
            
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class AltClip(Clip):
    def __init__(self, device, model_name="BAAI/AltCLIP"):
        super(AltClip, self).__init__(device)
        self.model = AltCLIPModel.from_pretrained(model_name).to(device)
        self.processor = AltCLIPProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt", padding=True
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class AlignClip(Clip):
    def __init__(self, device, model_name="kakaobrain/align-base"):
        super(AlignClip, self).__init__(device)
        self.model = AlignModel.from_pretrained(model_name).to(device)
        self.processor = AlignProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt"
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class ViTOpenAIClip(Clip):
    def __init__(
        self,
        device,
        base_name="ViT-B-32",
        pretrained="laion2b_s34b_b79k",
    ):
        super(ViTOpenAIClip, self).__init__(device)
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=base_name, pretrained=pretrained
        )
        self.model = self.model.to(device)
        self.tokenizer = open_clip.get_tokenizer(base_name)

    def forward(self, image, text_list):
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        text = self.tokenizer(text_list).to(self.device)
        with torch.no_grad():
            image_features = self.model.encode_image(image)
            text_features = self.model.encode_text(text)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        return text_probs.cpu().numpy()

In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.file_list = [
            os.path.join(folder_path, f)
            for f in sorted(os.listdir(folder_path))
            if os.path.isfile(os.path.join(folder_path, f))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        return self.file_list[idx], image


class ImageCaptionDataset(ImageFolderDataset):
    def __init__(self, folder_path, captions_file, transform=None):
        super(ImageCaptionDataset, self).__init__(folder_path, transform)
        self.captions = pd.read_csv(captions_file, sep=",")
        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        image_name = self.file_list[idx].split("/")[-1]
        caption = self.captions[self.captions["image_name"] == image_name]
        caption = caption["caption"].values[0]
        return image, caption


class TextDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(captions_file, sep="|")

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return self.captions.iloc[idx]["caption"]


class PipeDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(
            captions_file,
        )

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return (
            self.captions.iloc[idx]["subject"],
            self.captions.iloc[idx]["object"],
            self.captions.iloc[idx]["activity"],
            self.captions.iloc[idx]["areas"],
        )


class Loader:
    @staticmethod
    def load(path, batch_size, tan_scale=False, shuffle=False):
        mean = [0.485, 0.456, 0.406] if not tan_scale else [0.5, 0.5, 0.5]
        std = [0.229, 0.224, 0.225] if not tan_scale else [0.5, 0.5, 0.5]
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )
        generated_dataset = ImageFolderDataset(path, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_captions(path, captions, batch_size, shuffle=False):
        transform = None
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
            ]
        )
        generated_dataset = ImageCaptionDataset(path, captions, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_texts(captions, batch_size, shuffle=False):
        text_dataset = TextDataset(captions)
        return DataLoader(text_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_pipe(captions, batch_size, shuffle=False):
        pipe_dataset = PipeDataset(captions)
        return DataLoader(pipe_dataset, batch_size=batch_size, shuffle=shuffle)

In [4]:
CONFIG = {
    "alt": {
        "models": [["BAAI/AltCLIP"]],
        "handler": AltClip,
    },

    "flava": {
        "models": [["facebook/flava-full"]],
        "handler": Flava,
    },


    "align": {"models": [["kakaobrain/align-base"]], "handler": AlignClip},
    "openai": {
        "models": [
        # ["ViT-B-32", "negCLIP.pt"], # first download and add negClip weights in this directory
        ["EVA01-g-14", "laion400m_s11b_b41k"],
        ["EVA02-L-14", "merged2b_s4b_b131k"],
        ["RN50x64", "openai"],
        ["ViT-B-16", "openai"],
        ["ViT-B-32", "openai"],
        ["ViT-L-14", "openai"],
        ["coca_ViT-B-32", "laion2b_s13b_b90k"],
        ["coca_ViT-L-14", "laion2b_s13b_b90k"],
        ],
        "handler": ViTOpenAIClip,
    },
}

In [5]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torchvision.transforms as transforms
import gc


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAMES = CONFIG


def tendency_experiment(loader, prompts):
    result = []
    for api in tqdm(MODEL_NAMES):
      handler = MODEL_NAMES[api]["handler"]
      pbar = tqdm(MODEL_NAMES[api]["models"])
      for args in pbar:
        pbar.set_description(f"{api}_{' '.join(args)}")
        model = handler(DEVICE, *args)
        total_prompts = prompts
        for label, image in loader:
          inputs = transforms.ToPILImage()(image[0])
          probs= model(inputs, total_prompts)[0]
          result.append([f"{api}_{' '.join(args)}", prompts[int(np.argmax(probs))], label, int(np.argmax(probs)) , list(probs)])
          # result.append([f"{api}_{' '.join(args)}", neg_prompt, label, prob_neg])
        del model
        torch.cuda.empty_cache()
        gc.collect()
    df = pd.DataFrame(result)
    df.columns = ['model', 'prompt', 'image', 'prob', 'probs']
    return df

In [6]:
nearActions = {
'Beading Earrings':[

    "Mixing paints",
    "Cutting fabric",
    "Sketching designs",
    "Pouring candles into molds",
    "Planting seeds in small pots",
    "Organizing supplies into labeled bins",
    "Binding handmade journals",
    "Assembling mechanical models",
    "Applying varnish",
    "Glazing ceramics",
    "Editing photos",
    "Programming electronic components",
    "Sanding down rough edges on sculptures",
    "Testing recipes",
    "Arranging a display shelf",
    "Setting up lighting for photography of crafts",
    "Ironing fabric pieces",
    "cutting paper",
    "Blending essential oils",
    "Tuning musical instruments"

]
,

'Choosing Dress':
[
    "Selecting accessories",
    "Applying makeup",
    "Styling hair",
    "Trying on different shoes",
    "Consulting a stylist",
    "Ironing clothes",
    "Laying out clothing options the night before",
    "Matching a bag to the outfit",
    "Choosing jewelry",
    "Applying perfume",
    "Attaching cufflinks",
    "Polishing shoes",
    "Taking selfies",
    "Browsing fashion magazines",
    "Packing an emergency kit",
    "Selecting a watch",
    "Planning outfits",
    "Visiting a tailor"
]
,
'Baking Bread':
[
    "Kneading pizza dough",
    "Preparing a pie crust",
    "Whisking eggs for an omelette",
    "Boiling pasta",
    "Roasting vegetables",
    "Grilling meat",
    "Steaming rice",
    "Chopping vegetables",
    "Blending a smoothie",
    "Frying pancakes",
    "Simmering a stew",
    "Glazing pastries",
    "Decorating cupcakes",
    "Melting chocolate",
    "Canning fruits",
    "Marinating chicken for baking",
    "Whipping cream for toppings",
    "Measuring spices for a recipe"
]
,
'Picking Flower':
[
    "Planting vegetable seeds",
    "Watering garden plants",
    "Harvesting vegetables or fruits",
    "Applying mulch",
    "Inspecting plants for pests",
    "Fertilizing soil for better plant growth",
    "Setting up a bird feeder",
    "Building a compost bin",
    "Repotting houseplants",
    "Creating a butterfly garden",
    "Laying down stepping stones",
    "Installing a water feature",
    "Taking nature photographs",
    "Drawing outdoor landscapes",
    "Hiking in a nature reserve",
    "Participating in a community clean-up",
    "Observing wildlife in their natural habitat"
]
,
'Holding Baby'
:
[   
    "Arranging playdates",
    "Cooking meals",
    "Cleaning the living spaces",
    "Watering indoor plants",
    "Setting up educational toys",
    "Crafting in DIY projects",
    "Listening to music",
    "Making grocery lists",
    "Decorating the nursery's rooms",
    "Learning new parenting techniques online",
    "Attending virtual meetings",
    "Exercising at home",
    "Sewing or mending clothes",
    "Baking treats",
    "Starting a home garden",
    "Practicing meditation",
    "Reading books",
    "Organizing digital photos",
    "Writing in a journal",
    "Researching family-friendly activities"
    ]
,
'Leading Team':
[
    "Compiling financial reports",
    "Designing marketing materials",
    "Developing software",
    "Researching industry trends",
    "Managing client accounts",
    "Organizing files for easy access",
    "Conducting workshops",
    "Implementing cybersecurity measures",
    "Negotiating contracts with suppliers",
    "Administering employee benefits",
    "Maintaining the company website",
    "Ensuring workplace health and safety standards",
    "Coordinating travel arrangements for business trips",
    "Facilitating customer service",
    "executing product launches",
    "Conducting quality assurance tests",
    "Writing grant applications",
    "Managing supply chains",
    "Creating internal communication strategies",   
]
,
'Holding Gun':
[
    "Setting up safety barriers",
    "Organizing a first aid",
    "Conducting environmental conservation efforts",
    "Mapping out the area for navigation training",
    "Setting up communication equipment",
    "Preparing survival kits",
    "Teaching camouflage techniques",
    "Practicing non-verbal signals",
    "Setting up night-vision equipment",
    "Assembling and disassembling gear",
    "Organizing logistics and supplies",
    "Training with non-lethal tactics",
    "Learning about local flora",
    "Participating in fitness",
    "Practicing hand-to-hand combat techniques",
    "Conducting search and rescue simulations",
    "Engaging in team-building exercises",
    "Reviewing operational plans and objectives",
    "Cleaning the area post-activity"

]
,
'Drinking Beer':
[
    "Sampling different types of cheese",
    "Hosting a potluck dinner",
    "Playing board games",
    "Listening to live music",
    "Attending a mixology class",
    "Watching a sports game",
    "Participating in a pub quiz",
    "Exploring non-alcoholic craft beverages",
    "Joining a book club meeting",
    "Taking a brewery tour",
    "Organizing a backyard barbecue",
    "Engaging in a friendly cooking competition",
    "Taking part in a community art project",
    "Watching a movie",
    "Practicing home brewing of kombucha",
    "Attending a workshop",
    "Playing outdoor games",
    "Visiting a food festival",
    "Exploring local history"

]
,
'Climbing Tree'
:
[
    "Identifying different species of plants",
    "Observing wildlife",
    "Practicing nature photography",
    "Collecting fallen leaves",
    "Participating in a bird watching event",
    "Walking or hiking on nature trails",
    "Sketching landscapes",
    "Engaging in a guided nature tour",
    "Building a small campfire",
    "Camping overnight in a tent",
    "Practicing wilderness survival skills",
    "Foraging for edible plants under guidance",
    "Participating in a forest clean-up day",
    "Installing a rope swing",
    "Conducting a scientific study of the local ecosystem",
    "practicing yoga",
    "Mapping a new hiking route",
    "Creating land art with natural materials"
]
,
'Catching Fish'
:
[
    "Identifying aquatic plant species",
    "Observing waterfowl",
    "Collecting shells along the shore",
    "Taking water samples",
    "Practicing water safety",
    "Photographing landscapes",
    "Kayaking on a lake",
    "Participating in a beach clean-up",
    "Building sandcastles",
    "Exploring tide pools",
    "Snorkeling",
    "Hiking along coastal",
    "Setting up a picnic near the water",
    "Participating in a guided eco-tour",
    "Drawing outdoor scenes",
    "Camping near a body of water",
    "Watching the sunrise over the water",
    "navigating a boat"
]


}

In [7]:
def make_prompt(prompt_list):
    man_single_list = []
    man_double_list = []
    woman_single_list = []
    woman_double_list = []
    for action in prompt_list:
        man_single_list.append(f'a man is {action}')
        man_double_list.append(f'a man is {action} and a woman is in the scene')
        woman_single_list.append(f'a woman is {action}')
        woman_double_list.append(f'a woman is {action} and a man is in the scene')
    return man_single_list , man_double_list , woman_single_list , woman_double_list




In [ ]:

sourcePath = '../../phaze1/Phaze-1'
category1 = 'Beading Earrings'
action1 = 'beading earrings'

print(f'processing category : {category1}')
print()
print()

prompt_list = nearActions[category1]
prompt_list = [action1] + prompt_list 
completed_prompt = make_prompt(prompt_list) 

loader = Loader.load(f'{sourcePath}/{category1}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category1}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category1}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_man_2_binding.csv', index=False)









category2 = 'Choosing Dress'
action2 = 'choosing a dress'

print(f'processing category : {category2}')
print()
print()

prompt_list = nearActions[category2]
prompt_list = [action2] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category2}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category2}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category2}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_man_2_binding.csv', index=False)











category3 = 'Climbing Tree'
action3 = 'climbing a tree'

print(f'processing category : {category3}')
print()
print()


prompt_list = nearActions[category3]
prompt_list = [action3] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category3}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category3}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category3}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_man_2_binding.csv', index=False)











category4 = 'Holding Gun'
action4 = 'holding a gun'

print(f'processing category : {category4}')
print()
print()


prompt_list = nearActions[category4]
prompt_list = [action4] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category4}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category4}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category4}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_man_2_binding.csv', index=False)
















category5 = 'Leading Team'
action5 = 'leading a team'


print(f'processing category : {category5}')
print()
print()


prompt_list = nearActions[category5]
prompt_list = [action5] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category5}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category5}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category5}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_man_2_binding.csv', index=False)









category6 = 'Picking Flower'
action6 = 'picking a flower'


print(f'processing category : {category6}')
print()
print()

prompt_list = nearActions[category6]
prompt_list = [action6] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category6}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category6}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category6}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_man_2_binding.csv', index=False)









category7 = 'Holding Baby'
action7 = 'holding a baby'


print(f'processing category : {category7}')
print()
print()


prompt_list = nearActions[category7]
prompt_list = [action7] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category7}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category7}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category7}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_man_2_binding.csv', index=False)









category8 = 'Drinking Beer'
action8 = 'drinking bear'

print(f'processing category : {category8}')
print()
print()


prompt_list = nearActions[category8]
prompt_list = [action8] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category8}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category8}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category8}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_man_2_binding.csv', index=False)








category9 = 'Baking Bread'
action9 = 'baking bread'

print(f'processing category : {category9}')
print()
print()


prompt_list = nearActions[category9]
prompt_list = [action9] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category9}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category9}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category9}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_man_2_binding.csv', index=False)








category10 = 'Catching Fish'
action10 = 'catching fish'


print(f'processing category : {category10}')
print()
print()


prompt_list = nearActions[category10]
prompt_list = [action10] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category10}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category10}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category10}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_man_2_binding.csv', index=False)













